In [1]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Any

import numpy as np

from qqe.circuit.spec import CircuitSpec, GateSpec
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

from qqe.backend import QuimbBackend
from qqe.circuit.families import (
    CliffordBrickwork,
    HaarBrickwork,
    QuansistorBrickwork,
    RandomCircuit,
)
from qqe.circuit.gates import clifford_recipe_unitary
from qqe.experiments.plotting import plot_pennylane_circuit
from qqe.GNN import circuit_spec_to_nx_dag
from qqe.circuit.patterns import to_qasm


In [2]:
n_qubits = 8
n_layers = 30
seed = 42
family = "clifford"

In [3]:
family_registry = {
    "haar": HaarBrickwork,
    "clifford": CliffordBrickwork,
    "quansistor": QuansistorBrickwork,
    "random": RandomCircuit,
}

In [4]:
d_q = ["IN", "OUT", "RX", "RY", "RZ", "CX", "I", "H", "S", "T", "HAAR", "QX", "QY"]

In [5]:
d = len(d_q) + n_qubits 

In [6]:
circuit = family_registry[family]()
circuit_spec = circuit.make_spec(
    n_qubits=n_qubits,
    n_layers=n_layers,
    d=2,
    seed=seed,
)
gates = circuit_spec.gates

In [7]:
qasm = to_qasm(circuit_spec, gates)

In [8]:
qc = QuantumCircuit.from_qasm_str(qasm)
dag = circuit_to_dag(qc)

In [9]:
node_embedding = []
header = d_q + [f"q_{i}" for i in range(n_qubits)]
node_embedding.append(header)
print(node_embedding)

[['IN', 'OUT', 'RX', 'RY', 'RZ', 'CX', 'I', 'H', 'S', 'T', 'HAAR', 'QX', 'QY', 'q_0', 'q_1', 'q_2', 'q_3', 'q_4', 'q_5', 'q_6', 'q_7']]


In [10]:
def build_qubit_index(qc):
    # robust across qiskit versions
    return {q: qc.find_bit(q).index for q in qc.qubits}

def gate_mapping(name: str) -> str:
    n = name.lower()
    mapping = {
        "rx": "RX",
        "ry": "RY",
        "rz": "RZ",
        "cx": "CX",
        "ry": "RY",
        "rz": "RZ",
        "cx": "CX",
        "id": "I",
        "i": "I",
        "h": "H",
        "s": "S",
        "t": "T",
    }
    return mapping.get(n, n.upper())

def feature_dim(n_qubits: int) -> int:
    return len(d_q) + n_qubits

In [11]:
def encode_node(vocab_name: str, wires: list[int], *, n_qubits: int) -> np.ndarray:
    x = np.zeros(feature_dim(n_qubits), dtype=np.float32)

    # gate type one-hot
    if vocab_name in d_q:
        x[d_q.index(vocab_name)] = 1.0

    # qubit participation one-hot
    off = len(d_q)
    for w in wires:
        x[off + w] = 1.0

    return x

def iter_all_nodes_including_io(dag, qc):
    """
    Yields tuples: (node_obj, vocab_name, wires[int])
    Order: IN nodes (q0..), op nodes (topological), OUT nodes (q0..)
    """
    q_index = build_qubit_index(qc)

    # --- IN nodes ---
    # dag.input_map: {Qubit: InNode}
    for q in qc.qubits:
        in_node = dag.input_map[q]
        wires = [q_index[q]]
        yield in_node, "IN", wires

    # --- OP nodes ---
    for op_node in dag.topological_op_nodes():
        wires = [q_index[q] for q in op_node.qargs]
        vocab = gate_mapping(op_node.name)
        yield op_node, vocab, wires

    # --- OUT nodes ---
    for q in qc.qubits:
        out_node = dag.output_map[q]
        wires = [q_index[q]]
        yield out_node, "OUT", wires

def encode_all_nodes_with_io(dag, qc):
    """
    Returns:
      X: (num_nodes_total, D)
      node_names: list[str]
      node_wires: list[list[int]]
    """
    n_qubits = qc.num_qubits
    feats = []
    node_names = []
    node_wires = []

    for _, vocab, wires in iter_all_nodes_including_io(dag, qc):
        feats.append(encode_node(vocab, wires, n_qubits=n_qubits))
        node_names.append(vocab)
        node_wires.append(wires)

    X = np.stack(feats, axis=0)
    return X, node_names, node_wires

In [12]:
X, node_wires, node_names = encode_all_nodes_with_io(dag, qc)

print("X shape:", X.shape)                 # (num_nodes, feature_dim)
print("first node:", node_names[0], node_wires[0], X[0][:len(d_q)])

X shape: (331, 21)
first node: [0] IN [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
import pandas as pd
df = pd.DataFrame(X, columns=d_q + [f"q_{i}" for i in range(qc.num_qubits)])

In [14]:
df

,IN,OUT,RX,RY,RZ,CX,I,H,S,T,...,QX,QY,q_0,q_1,q_2,q_3,q_4,q_5,q_6,q_7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
327,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
328,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
329,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
fig = qc.draw("mpl", vertical_compression="low", fold=-1)
fig.savefig(f"circuit_{family}.png", dpi=200, bbox_inches="tight")

In [16]:
# from qiskit.visualization import dag_drawer

# dag_drawer(dag, filename=f"dag_{family}.png", style="color")

In [17]:
A = np.zeros((n_qubits, n_qubits))

In [18]:
A

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [19]:
dag.output_map

{<Qubit register=(8, "q"), index=0>: DAGOutNode(wire=<Qubit register=(8, "q"), index=0>),
 <Qubit register=(8, "q"), index=1>: DAGOutNode(wire=<Qubit register=(8, "q"), index=1>),
 <Qubit register=(8, "q"), index=2>: DAGOutNode(wire=<Qubit register=(8, "q"), index=2>),
 <Qubit register=(8, "q"), index=3>: DAGOutNode(wire=<Qubit register=(8, "q"), index=3>),
 <Qubit register=(8, "q"), index=4>: DAGOutNode(wire=<Qubit register=(8, "q"), index=4>),
 <Qubit register=(8, "q"), index=5>: DAGOutNode(wire=<Qubit register=(8, "q"), index=5>),
 <Qubit register=(8, "q"), index=6>: DAGOutNode(wire=<Qubit register=(8, "q"), index=6>),
 <Qubit register=(8, "q"), index=7>: DAGOutNode(wire=<Qubit register=(8, "q"), index=7>)}

In [20]:
dag.count_ops

<function DAGCircuit.count_ops(*, recurse=True)>

In [21]:
# Build a mapping from DAG nodes to unique indices
node_to_index = {}
current_index = 0

# Map input nodes
for node in dag.input_map.values():
    node_to_index[node] = current_index
    current_index += 1

# Map op nodes
for node in dag.op_nodes():
    node_to_index[node] = current_index
    current_index += 1

# Map output nodes
for node in dag.output_map.values():
    node_to_index[node] = current_index
    current_index += 1

# Total number of nodes
N = current_index

# Initialize adjacency matrix
A = np.zeros((N, N), dtype=int)

# Fill adjacency matrix based on edges
for edge in dag.edges():
    node_a = edge[0]  # source node
    node_b = edge[1]  # target node

    index_a = node_to_index[node_a]
    index_b = node_to_index[node_b]

    A[index_a][index_b] = 1

print(f"Adjacency matrix shape: {A.shape}")
print(f"Total nodes: {N}")
print(f"Total edges: {np.sum(A)}")

Adjacency matrix shape: (331, 331)
Total nodes: 331
Total edges: 428


In [22]:
A_df = pd.DataFrame(A)

In [23]:
A_df

,0,1,2,3,4,5,6,7,8,9,...,321,322,323,324,325,326,327,328,329,330
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_df = pd.DataFrame(X, columns=d_q + [f"q_{i}" for i in range(qc.num_qubits)])

In [25]:
X_df

,IN,OUT,RX,RY,RZ,CX,I,H,S,T,...,QX,QY,q_0,q_1,q_2,q_3,q_4,q_5,q_6,q_7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
327,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
328,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
329,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
from qqe.GNN.encoder import qasm_to_pyg_graph

In [27]:
graph_data, gate_counts = qasm_to_pyg_graph(
    qasm_str=qasm,
    n_bins=50,
    family=family,
    global_feature_variant="binned",
)


In [28]:
print(f"------ Family: {family} -----")
print(graph_data)
print(gate_counts)

------ Family: clifford -----
Data(x=[331, 27], edge_index=[2, 428], num_qubits=8, global_features=[7])
{'I_count': 64, 'H_count': 66, 'S_count': 80, 'T_count': 0, 'CNOT_count': 105}


In [29]:
for row in graph_data.x:
    print(row)

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0.])


In [31]:
from qqe.GNN.physics_aware_NN import GNN

,0,1,2,3,4,5,6,7,8,9,...,402,403,404,405,406,407,408,409,410,411
0,0,1,2,3,4,5,6,7,15,14,...,324,323,319,327,327,322,329,329,328,328
1,8,9,10,11,12,13,14,15,16,16,...,329,329,330,331,332,333,334,335,336,337
